In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score, mean_squared_log_error, mean_squared_error,mean_absolute_error
import datetime as dt
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import preparing_data as F
import ml_module as ml

In [2]:
#data=pd.read_pickle("./dataframe/df_20210912_133401_ONLY_RISKY_EVENTS-6-withFE.pkl")
#data=pd.read_pickle("./dataframe/df_20210914_003013_equally_distributed.pkl")
data=pd.read_pickle("./dataframe/df_only_risky_events.pkl")
data.reset_index(inplace=True)
data.drop(['index'], inplace=True, axis=1)
print(data.shape)
data.head()

(272, 86)


,__time_to_tca,MISS_DISTANCE,RELATIVE_SPEED,RELATIVE_POSITION_R,RELATIVE_POSITION_T,RELATIVE_POSITION_N,RELATIVE_VELOCITY_R,RELATIVE_VELOCITY_T,RELATIVE_VELOCITY_N,COLLISSION_PROBABILITY,...,PC_mavg_1,PC_trend_1,PC_trend_3,PC_gradient_1,PC_gradient_3,MD_mavg_1,MD_trend_1,MD_trend_3,MD_gradient_1,MD_gradient_3
0,0.832257,2244.0,11172.0,87.9,-1474.9,1689.9,-201.4,-8424.3,-7336.0,-3.418392,...,-3.475175,0.132439,0.590914,0.354472,0.579354,2181.333333,47.0,17.0,125.795356,16.667421
1,0.580855,966.0,212.0,191.0,-944.8,-68.9,33.8,-8.4,209.3,-3.711974,...,-3.981494,0.310256,0.575376,0.747481,0.536846,1131.000000,-201.0,-326.0,-484.256505,-304.169574
2,0.189825,1102.0,212.0,193.3,-1083.1,-75.0,33.9,-8.4,209.3,-3.913996,...,-3.882734,-0.202022,0.296282,-0.516640,0.270366,1078.333333,136.0,-158.0,347.799165,-144.179478
3,0.994637,456.0,11707.0,-22.1,288.8,353.3,-10.2,-9077.7,7392.6,-2.055073,...,-2.841298,1.280567,1.312874,3.634346,1.445698,709.666667,-577.0,124.0,-1637.570088,136.545083
4,0.759012,629.0,11707.0,-18.7,395.3,489.9,-10.3,-9077.7,7392.6,-2.649558,...,-2.680091,-0.594485,0.483623,-2.523011,0.486607,706.000000,173.0,-11.0,734.217036,-11.067877


In [3]:
#print(list(data.columns))

In [4]:
train, test = train_test_split(data, test_size=0.30, random_state=42)

In [5]:
print("Train dataframe dimension {} x {}".format(train.shape[0],train.shape[1]))
print("Test dataframe dimension {} x {}".format(test.shape[0],test.shape[1]))

Train dataframe dimension 190 x 86
Test dataframe dimension 82 x 86


In [6]:
Y_train = train["COLLISSION_PROBABILITY"]
X_train= train.drop(["COLLISSION_PROBABILITY"], axis=1)
Y_test = test["COLLISSION_PROBABILITY"]
X_test= test.drop(["COLLISSION_PROBABILITY"], axis=1)


In [7]:
X = X_train
y = Y_train

In [8]:
#model=ml.create_and_validate_model(X, y, X_test, Y_test,
 #                       init_iter=5, n_iters=1500, random_state=77, seed = 101,num_iterations=300)

|   iter    |  target   | baggin... | featur... | learni... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.848    |  0.9757   |  0.6138   |  0.03793  |  8.115    |  12.18    |  0.07901  |  79.57    |
|  2        |  0.8      |  0.8623   |  0.2922   |  0.02773  |  10.2     |  27.88    |  0.08383  |  95.31    |
|  3        |  0.8222   |  0.7888   |  0.3248   |  0.03557  |  10.38    |  11.43    |  0.07496  |  87.14    |
|  4        |  0.5445   |  0.7527   |  0.1395   |  0.01533  |  7.534    |  28.78    |  0.007313 |  85.91    |
|  5        |  0.7843   |  0.8093   |  0.2216   |  0.02779  |  10.55    |  10.9     |  0.08247  |  76.4     |
|  6        |  0.8548   |  1.0      |  0.9      |  0.05     |  14.18    |  15.54    |  0.1      |  82.26    |
|  7        |  0.8322   |  0.9823   |  0.4821   |  0.04194  |  13.12    |  26.09    |  0.1      |  104.2    |
|  8      

In [9]:
# ml.create_and_validate_model(X, y,
#                         init_iter=5, n_iters=1500, random_state=77, seed = 101,num_iterations=300,evalm="lgb_adjusted_r2")

In [10]:
# ml.create_and_validate_model(X, y,
#                         init_iter=5, n_iters=1500, random_state=77, seed = 101,num_iterations=300,evalm="lgb_rmse",hp_metric="rmse")

In [18]:
aux=ml.load_and_validate_model("param_lgb_r2_only_risky_events",X,y,X_test,Y_test)
df_r2=aux[1]
aux=ml.load_and_validate_model("param_lgb_adjusted_r2_only_risky_events",X,y,X_test,Y_test)
df_r2_adj=aux[1]

------------------------ OPTIMAL PARAMETERS ------------------------
{'bagging_fraction': 0.8693796326739299, 'feature_fraction': 0.8168553631386558, 'learning_rate': 0.0464041915708064, 'max_depth': 13.009239099967989, 'min_child_weight': 10.19191247052637, 'min_split_gain': 0.004793459623540061, 'num_leaves': 93.8608845306346, 'num_iterations': 300, 'n_estimators': 1000}
-------------------------------------------------------------------
The r2 of prediction is: 0.9065216981160615
The MSE of prediction is: 0.05162276773487457
The RMSE of prediction is: 0.22720644298715337
The MAE of prediction is: 0.15151940051952298
------------------------ OPTIMAL PARAMETERS ------------------------
{'bagging_fraction': 0.9747766114833514, 'feature_fraction': 0.8946608757396138, 'learning_rate': 0.048335934814995346, 'max_depth': 11.685910466019436, 'min_child_weight': 13.355916840770721, 'min_split_gain': 0.00795711246669661, 'num_leaves': 108.61067784693134, 'num_iterations': 300, 'n_estimators':

In [11]:
#df=model[1]

In [19]:
df=df_r2

In [20]:
df.head()

,y_true,y_predicted,y_true_10,y_predicted_10
0,-4.558148,-4.622216,0.000028,0.000024
1,-5.849858,-5.861389,0.000001,0.000001
2,-4.882729,-5.137882,0.000013,0.000007
3,-5.337053,-5.338532,0.000005,0.000005
4,-5.325139,-5.367577,0.000005,0.000004


In [21]:
df[df["y_true_10"]>0.00001]

,y_true,y_predicted,y_true_10,y_predicted_10
0,-4.558148,-4.622216,0.000028,0.000024
2,-4.882729,-5.137882,0.000013,0.000007
7,-4.982967,-5.181214,0.000010,0.000007
13,-4.387110,-4.220966,0.000041,0.000060
19,-4.407934,-4.523292,0.000039,0.000030
22,-4.201418,-4.336120,0.000063,0.000046
23,-4.775985,-4.717772,0.000017,0.000019
25,-3.254067,-3.673389,0.000557,0.000212
26,-2.528122,-3.614213,0.002964,0.000243
29,-4.119529,-4.037598,0.000076,0.000092


In [22]:
df[df["y_true_10"]>0.0001]

,y_true,y_predicted,y_true_10,y_predicted_10
25,-3.254067,-3.673389,0.000557,0.000212
26,-2.528122,-3.614213,0.002964,0.000243
35,-3.716021,-3.677709,0.000192,0.000210
43,-3.501965,-3.681524,0.000315,0.000208
50,-3.753994,-3.698844,0.000176,0.000200
60,-3.513428,-3.694179,0.000307,0.000202
63,-3.812197,-4.154395,0.000154,0.000070
70,-3.499215,-3.578884,0.000317,0.000264


In [24]:
df_r2_adj[df_r2_adj["y_true_10"]>0.0001]

,y_true,y_predicted,y_true_10,y_predicted_10
25,-3.254067,-3.726029,0.000557,0.000188
26,-2.528122,-3.528397,0.002964,0.000296
35,-3.716021,-3.687190,0.000192,0.000205
43,-3.501965,-3.668473,0.000315,0.000215
50,-3.753994,-3.663359,0.000176,0.000217
60,-3.513428,-3.686435,0.000307,0.000206
63,-3.812197,-4.124028,0.000154,0.000075
70,-3.499215,-3.569816,0.000317,0.000269


In [25]:
pr2=df[df["y_true_10"]>0.0001]
pr2_adj=df_r2_adj[df_r2_adj["y_true_10"]>0.0001]

In [27]:
pd.concat([pr2,pr2_adj],axis=1)

,y_true,y_predicted,y_true_10,y_predicted_10,y_true,y_predicted,y_true_10,y_predicted_10
25,-3.254067,-3.673389,0.000557,0.000212,-3.254067,-3.726029,0.000557,0.000188
26,-2.528122,-3.614213,0.002964,0.000243,-2.528122,-3.528397,0.002964,0.000296
35,-3.716021,-3.677709,0.000192,0.000210,-3.716021,-3.687190,0.000192,0.000205
43,-3.501965,-3.681524,0.000315,0.000208,-3.501965,-3.668473,0.000315,0.000215
50,-3.753994,-3.698844,0.000176,0.000200,-3.753994,-3.663359,0.000176,0.000217
60,-3.513428,-3.694179,0.000307,0.000202,-3.513428,-3.686435,0.000307,0.000206
63,-3.812197,-4.154395,0.000154,0.000070,-3.812197,-4.124028,0.000154,0.000075
70,-3.499215,-3.578884,0.000317,0.000264,-3.499215,-3.569816,0.000317,0.000269


In [29]:
data = [pr2["y_true_10"], pr2["y_predicted_10"],pr2_adj["y_predicted_10"]]
headers = ["PC_true", "PC_predic_r2","PC_predic_r2_ajus"]
df3 = pd.concat(data, axis=1, keys=headers)
df3

,PC_true,PC_predic_r2,PC_predic_r2_ajus
25,0.000557,0.000212,0.000188
26,0.002964,0.000243,0.000296
35,0.000192,0.000210,0.000205
43,0.000315,0.000208,0.000215
50,0.000176,0.000200,0.000217
60,0.000307,0.000202,0.000206
63,0.000154,0.000070,0.000075
70,0.000317,0.000264,0.000269


In [30]:
mylist=["ZONA DE MANIOBRA","ZONA DE MANIOBRA","ZONA DE MANIOBRA","ZONA DE MANIOBRA","ZONA DE MANIOBRA","ZONA DE MANIOBRA","ZONA DE ALERTA","ZONA DE MANIOBRA"]
df3["label"]=np.array(mylist)

In [31]:
df3

,PC_true,PC_predic_r2,PC_predic_r2_ajus,label
25,0.000557,0.000212,0.000188,ZONA DE MANIOBRA
26,0.002964,0.000243,0.000296,ZONA DE MANIOBRA
35,0.000192,0.000210,0.000205,ZONA DE MANIOBRA
43,0.000315,0.000208,0.000215,ZONA DE MANIOBRA
50,0.000176,0.000200,0.000217,ZONA DE MANIOBRA
60,0.000307,0.000202,0.000206,ZONA DE MANIOBRA
63,0.000154,0.000070,0.000075,ZONA DE MONITOREO
70,0.000317,0.000264,0.000269,ZONA DE MANIOBRA
